# Binary Classification Lab

## Dataset Review

The Adult dataset we are going to use is publicly available at the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Adult).
This data derives from census data, and consists of information about 48842 individuals and their annual income.
We will use this information to predict if an individual earns **<=50K or >50k** a year.
The dataset is rather clean, and consists of both numeric and categorical variables.

Attribute Information:

- age: continuous
- workclass: Private,Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked
- fnlwgt: continuous
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc...
- education-num: continuous
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent...
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners...
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black
- sex: Female, Male
- capital-gain: continuous
- capital-loss: continuous
- hours-per-week: continuous
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany...

Target/Label: - <=50K, >50K

In [ ]:
# Set up the environment for using pyspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
spark = SparkSession\
        .builder\
        .appName("Income Prediction")\
        .getOrCreate()
sc = spark.sparkContext

## Load Data

Load data from the datasets directory (agent.csv)

In [ ]:
dataset = spark.read.format('csv').options(header='true', inferSchema='true').load('../datasets/agent.csv')

In [ ]:
cols = dataset.columns

In [ ]:
dataset.limit(5).toPandas().head()

## Preprocess Data

Since we are going to try algorithms like Logistic Regression, we will have to convert the categorical variables in the dataset into numeric variables.
There are 2 ways we can do this.

* Category Indexing

  This is basically assigning a numeric value to each category from {0, 1, 2, ...numCategories-1}.
  This introduces an implicit ordering among your categories, and is more suitable for ordinal variables (eg: Poor: 0, Average: 1, Good: 2)

* One-Hot Encoding

  This converts categories into binary vectors with at most one nonzero value (eg: (Blue: [1, 0]), (Green: [0, 1]), (Red: [0, 0]))

In this dataset, we have ordinal variables like education (Preschool - Doctorate), and also nominal variables like relationship (Wife, Husband, Own-child, etc).
For simplicity's sake, we will use One-Hot Encoding to convert all categorical variables into binary vectors.
It is possible here to improve prediction accuracy by converting each categorical column with an appropriate method.

Here, we will use a combination of [StringIndexer] and [OneHotEncoderEstimator] to convert the categorical variables.
The `OneHotEncoderEstimator` will return a [SparseVector].

Since we will have more than 1 stage of feature transformations, we use a [Pipeline] to tie the stages together.
This simplifies our code.

[StringIndexer]: http://spark.apache.org/docs/latest/ml-features.html#stringindexer
[OneHotEncoderEstimator]: https://spark.apache.org/docs/latest/ml-features.html#onehotencoderestimator
[SparseVector]: https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.linalg.SparseVector
[Pipeline]: http://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.Pipeline

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
categoricalColumns = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "sex", "native_country"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    # encoder = OneHotEncoderEstimator(inputCol=categoricalCol + "Index", outputCol=categoricalCol + "classVec")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    
    # Add stages.  These are not run here, but will run all at once later on.
    stages += [stringIndexer, encoder]

The above code basically indexes each categorical column using the `StringIndexer`,
and then converts the indexed categories into one-hot encoded variables.
The resulting output has the binary vectors appended to the end of each row.

We use the `StringIndexer` again to encode our labels to label indices.

In [ ]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol="income", outputCol="label")
stages += [label_stringIdx]

Use a `VectorAssembler` to combine all the feature columns into a single vector column.
This includes both the numeric columns and the one-hot encoded binary vector columns in our dataset.

In [ ]:
# Transform all features into a vector using VectorAssembler
numericCols = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

Run the stages as a Pipeline. This puts the data through all of the feature transformations we described in a single call.

In [ ]:
from pyspark.ml.classification import LogisticRegression
  
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(dataset)
preppedDataDF = pipelineModel.transform(dataset)

In [ ]:
# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = preppedDataDF.select(selectedcols)

In [ ]:
dataset.limit(5).toPandas().head()

## Create Training and Test sets and print their shape

In [ ]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=2345)
print(trainingData.count())
print(testData.count())

## Fit and Evaluate Models

We are now ready to try out some of the Binary Classification algorithms available in the Pipelines API.

Out of these algorithms, the below are also capable of supporting multiclass classification with the Python API:

- Logistic Regression
- Decision Tree Classifier
- Random Forest Classifier


We use the `BinaryClassificationEvaluator` to evaluate our models, which uses [areaUnderROC] as the default metric.

[areaUnderROC]: https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve

## Logistic Regression

You can read more about [Logistic Regression] from the [classification and regression] section of MLlib Programming Guide.
In the Pipelines API, we are now able to perform Elastic-Net Regularization with Logistic Regression, as well as other linear methods.

[classification and regression]: https://spark.apache.org/docs/latest/ml-classification-regression.html
[Logistic Regression]: https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [ ]:
# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)

In [ ]:
# View model's predictions and probabilities of each prediction class
# You can select any columns in the above schema to view as well. For example's sake we will choose age & occupation
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
selected.show()

We can use ``BinaryClassificationEvaluator`` to evaluate our model. We can set the required column names in `rawPredictionCol` and `labelCol` Param and the metric in `metricName` Param.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

Note that the default metric for the ``BinaryClassificationEvaluator`` is ``areaUnderROC``

In [ ]:
evaluator.getMetricName()

The evaluator currently accepts 2 kinds of metrics - areaUnderROC and areaUnderPR.
We can set it to areaUnderPR by using evaluator.setMetricName("areaUnderPR").

Now we will try tuning the model with the ``ParamGridBuilder`` and the ``CrossValidator``.

If you are unsure what params are available for tuning, you can use ``explainParams()`` to print a list of all params and their definitions.

In [ ]:
print(lr.explainParams())

<font color = 'black'>
    <h2>Implement ParamGridBuilder and CrossValidator using following parameters:</h2>

1. regParam - 0.01, 0.5, 2.0
2. elasticNetParam - 0.0, 0.5, 1.0
3. maxIter - 1, 5, 10

Set numFolds to 5
</font>

<span style="font-family:times, serif; font-size:14pt; font-style:italic">
As we indicate 3 values for regParam, 3 values for maxIter, and 2 values for elasticNetParam,
this grid will have 3 x 3 x 3 = 27 parameter settings for CrossValidator to choose from.
We will create a 5-fold cross validator.
</span>

can also access the model's feature weights and intercepts easily

## Decision Trees

You can read more about [Decision Trees](http://spark.apache.org/docs/latest/mllib-decision-tree.html) in the Spark MLLib Programming Guide.
The Decision Trees algorithm is popular because it handles categorical
data and works out of the box with multiclass classification tasks.

<font color='tomato'>
    <h2>Implement Decision Tree Classifier for the our dataset</h2>
    1. Set initial maxDepth to 3 <br>
    2. Create model by applying fit to training data<br>
    3. Print number of nodes and depth of tree<br>
    4. Make predictions by using test data on the model<br>
    5. Apply the binary evaluator and evaluate the model<br>
</font>

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
# Create initial Decision Tree Model


We can extract the number of nodes in our decision tree as well as the
tree depth of our model.

In [ ]:
# Make predictions on test data using the Transformer.transform() method.


In [ ]:
# View model's predictions and probabilities of each prediction class


We will evaluate our Decision Tree model with
`BinaryClassificationEvaluator`.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


Entropy and the Gini coefficient are the supported measures of impurity for Decision Trees. This is ``Gini`` by default. Changing this value is simple, ``model.setImpurity("Entropy")``.

dt.getImpurity()

<font color = 'tomato'>
    <h2>Bonus - Implement ParamGridBuilder and CrossValidator using following parameters:</h2>

1. maxDepth - 1, 2, 6, 10
2. maxBins - 20, 40, 80
3. maxIter - 1, 5, 10

Set numFolds to 5
</font>

Now we will try tuning the model with the ``ParamGridBuilder`` and the ``CrossValidator``.

As we indicate 3 values for maxDepth and 3 values for maxBin, this grid will have 3 x 3 = 9 parameter settings for ``CrossValidator`` to choose from. We will create a 5-fold CrossValidator.

In [ ]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


In [ ]:
# Create 5-fold CrossValidator


In [ ]:
# Use test set to measure the accuracy of our model on new data


In [ ]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model


In [ ]:
# View Best model's predictions and probabilities of each prediction class


## Random Forest

Random Forests uses an ensemble of trees to improve model accuracy.
You can read more about [Random Forest] from the [classification and regression] section of MLlib Programming Guide.

[classification and regression]: https://spark.apache.org/docs/latest/ml-classification-regression.html
[Random Forest]: https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forests

<font color='tomato'>
    <h2>Implement Random Forest Classifier for the our dataset</h2>
    1. Create model by applying fit to training data<br>
    2. Make predictions by using test data on the model<br>
    3. Apply the binary evaluator and evaluate the model<br>
</font>

In [ ]:
from pyspark.ml.classification import RandomForestClassifier



In [ ]:
# Make predictions on test data using the Transformer.transform() method.


In [ ]:
# View model's predictions and probabilities of each prediction class


We will evaluate our Random Forest model with `BinaryClassificationEvaluator`.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator



<font color = 'tomato'>
    <h3>Bonus - Implement ParamGridBuilder and CrossValidator using following parameters:</h3>

1. maxDepth - 2, 4, 5
2. maxBins - 20, 60
3. numTrees - 5, 20

Set numFolds to 5
</font>

Now we will try tuning the model with the ``ParamGridBuilder`` and the ``CrossValidator``.

As we indicate 3 values for maxDepth, 2 values for maxBin, and 2 values for numTrees,
this grid will have 3 x 2 x 2 = 12 parameter settings for ``CrossValidator`` to choose from.
We will create a 5-fold ``CrossValidator``.

In [ ]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator



In [ ]:
# Create 5-fold CrossValidator


In [ ]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [ ]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model


In [ ]:
# View Best model's predictions and probabilities of each prediction class


## Make Predictions
As Random Forest gives us the best areaUnderROC value, we will use the bestModel obtained from Random Forest for deployment,
and use it to generate predictions on new data.
In this example, we will simulate this by generating predictions on the entire dataset.<br>
<font color = 'tomato'>Make sure the cross validation model is called cvModel otherwise following statement will error
</font>

In [ ]:
bestModel = cvModel.bestModel

In [ ]:
# Generate predictions for entire dataset
finalPredictions = bestModel.transform(dataset)

In [ ]:
# Evaluate best model
evaluator.evaluate(finalPredictions)

In this example, we will also look into predictions grouped by age and occupation.

In [ ]:
finalPredictions.createOrReplaceTempView("finalPredictions")

In an operational environment, analysts may use a similar machine learning pipeline to obtain predictions on new data, organize it into a table and use it for analysis or lead targeting.

In [ ]:
x1 = spark.sql(
"SELECT occupation, prediction, count(*) AS count \
FROM finalPredictions \
GROUP BY occupation, prediction \
ORDER BY occupation")
x1.show()

In [ ]:
x1 = spark.sql(
"SELECT age, prediction, count(*) AS count \
FROM finalPredictions \
GROUP BY age, prediction \
ORDER BY age")
x1.show()